# **Sentiment Analysis for major airlines in United States**  [2]

In [0]:
import pandas as pd
import numpy as np
from collections import Counter

#**Dataset** [3]

This dataset is of 14.5k tweets for each major airline in United States. The dataset consist of the following column classify positive, negative, and neutral tweets, followed by categorizing negative reasons (such as "bad service" or "Can't Tell").
This dataset can be used for sentimental analysis as well as in the field of Data Science.

I planned that I will use dataset for sentiment analysis and classify is the tweet a positive tweet or a negative tweet.

I think so there are many other datasets like IMDb movie review, Rotten Tamato revies, Yelp dataset  that we can use for classification and they are accessible on internet. 



In [0]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/Tweets.csv')
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [0]:
reviews = np.array(data['text'])[:14000]
labels = np.array(data['airline_sentiment'])[:14000]

In [0]:
data['text'].loc[101]

'@VirginAmerica why must a traveler miss a flight to Late Flight check a bag?  I missed my morning appointments and you lost my business. #sfo2lax'

In [0]:
data['airline_sentiment'].loc[101]

'negative'

**Text Preprocessing**

Text cleaning and preprocessing one of the major methods that need to carry out for NLP. The text that we use is a mess, they consist of slang,  its unstructured, grammar and syntax rule, which machine cannot understand. 
In order to machine to understand we preprocess the text data so that it is predictable and analyzable for machine. The preprocessing of the text data consist of following: 
*   Tokenization
*   Removing Stop Words
*   Lowercase the whole text data
*   Word Embedding
*   Removing the numbers
*   Removing punctuations
*   Removing white space
*   Padding the sentence









In [0]:
punctuation = '!"#$%&\'()*+,-./:;<=>?[\\]^_`{|}~'

# get rid of punctuation
reviews = 'separator'.join(reviews)
reviews = reviews.lower()
text = ''.join([c for c in reviews if c not in punctuation])

reviews_split = text.split('separator')
text = ' '.join(reviews_split)

words = text.split()

In [0]:
new_reviews = []
for review in reviews_split:
    review = review.split()
    new_text = []
    for word in review:
        if (word[0] != '@') & ('http' not in word) & (~word.isdigit()):
            new_text.append(word)
    new_reviews.append(new_text)

In [0]:
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

reviews_ints = []
for review in new_reviews:
    reviews_ints.append([vocab_to_int[word] for word in review])

In [0]:
encoded_labels = []
for label in labels:
    if label == 'neutral':
        encoded_labels.append(1)
    elif label == 'negative':
        encoded_labels.append(0)
    else:
        encoded_labels.append(1)

encoded_labels = np.asarray(encoded_labels)

In [0]:
def padding(reviews_ints, seq_length):
  # getting the correct rows x cols shape
  features = np.zeros((len(reviews_ints), seq_length), dtype=int)

  # for each review, I grab that review and 
  for i, row in enumerate(reviews_ints):
      features[i, -len(row):] = np.array(row)[:seq_length]
  
  return features

In [0]:
seq_length = 30
features = padding(reviews_ints, seq_length=seq_length)

In [0]:
split = 0.8

split_idx = int(len(features)*split)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

## print out the shapes of the resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: {}".format(train_x.shape), 
      "\nValidation set:{}".format(val_x.shape),
      "\nTest set: {}".format(test_x.shape))

			Feature Shapes:
Train set: (11200, 30) 
Validation set:(1400, 30) 
Test set: (1400, 30)


In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader

train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

batch_size = 50

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [0]:
data_iteration = iter(train_loader)
sample_x, sample_y = data_iteration.next()

In [0]:
device=torch.cuda.is_available()

if(device):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


# Q1) Why did you choose this architecture?



Ans: Reason behid choosing RNN-LSTM model is the data that I have choosen is made of sequence of words and RNN is known for short term dependencies, it also have feedback mechanism i.e. the current word can be dependent on previous word(s). While CNN do not have the feedback mechanism like RNN and CNN can be used only for extracting feature from dataset i.e. it can be better than RNN for extracting the meaning of the single word rather than the whole sentence. Even the RNN is known for short term dependencies it can store a small sentence, but if the dataset have a sentence of 30 words (I just aassumed that dataset may have the sentence with 30 words in it) RNN may lose the track of previous words. Hence I used LSTM so that it can store the whole sentence of 30 words.

# Q2) What else did you try?

Ans: I tried the RNN-GRU architecture becase GRU can be train faster than LSTM only on less data, they are simple and easy to modify. But if the data is huge and have long sequences then LSTM is better choice. In my case I had moderate amount of data (15 k tweets) on which GRU can work efficently. But there were some tweets that had a long sequence so I chose LSTM over GRU. Also I have worked on GRU last semeter for my CS440 class where I trained a chatbot using RNN-GRU netowrk and I wanted to try something else so this one was also one of the reason choosing LSTM over GRU. 

**I also tried using transformer but I failed to do so because its easy to understand but difficult to train. So I might give another try for tranformer in future.**

# Q3) Explain your loss function. Did it perform as well as you had hoped?

Ans: For loss function I used Binary Cross Entropy (aka log loss) because it is an binary classification problem. My LSTM model is returning the sigmoid value which is in the range 0 to 1. No, the loss function is performing quite bad because it's not linear.

# Q4)How could you improve your model?

Ans: I can improve my model using more dataset. The another approach is Transformer which I tried but was not able to build the network and I am thinking of giving it another try later in the semester.

In [0]:
import torch.nn as nn

class RNN(nn.Module):

  def __init__(self, vocab_size, output_size, embedding_dimension, hidden_dimension, n_layers, drop_prob=0.3):

    super(RNN, self).__init__()

    self.output_size = output_size
    self.n_layers = n_layers
    self.hidden_dimension = hidden_dimension
    self.embedding = nn.Embedding(vocab_size, embedding_dimension)
    self.lstm = nn.LSTM(embedding_dimension, hidden_dimension, n_layers, 
                        dropout=drop_prob, batch_first=True)

    self.dropout = nn.Dropout(0.3)
    self.fc = nn.Linear(hidden_dimension, output_size)
    self.sig = nn.Sigmoid()
      

  def forward(self, x, hidden):

    batch_size = x.size(0)
    x = x.long()
    embeds = self.embedding(x)
    lstm_out, hidden = self.lstm(embeds, hidden)
    lstm_out = lstm_out.contiguous().view(-1, self.hidden_dimension)

    out = self.dropout(lstm_out)
    out = self.fc(out)
    sig_out = self.sig(out)
    sig_out = sig_out.view(batch_size, -1)
    sig_out = sig_out[:, -1] # get last batch of labels
    
    # return last sigmoid output and hidden state
    return sig_out, hidden
  
  
  def init_hidden(self, batch_size):
    weight = next(self.parameters()).data
    
    if (device):
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dimension).zero_().cuda(),
              weight.new(self.n_layers, batch_size, self.hidden_dimension).zero_().cuda())
    else:
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dimension).zero_(),
                  weight.new(self.n_layers, batch_size, self.hidden_dimension).zero_())
    
    return hidden

In [0]:
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dimension = 200
hidden_dimension = 128
n_layers = 2

model = RNN(vocab_size, output_size, embedding_dimension, hidden_dimension, n_layers)

print(model)

RNN(
  (embedding): Embedding(16728, 200)
  (lstm): LSTM(200, 128, num_layers=2, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [0]:
lr=0.01

model_loss = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [0]:
import matplotlib.pyplot as plt
epochs = 10

counter = 0
print_every = 100
gradient_clip=5 

if(device):
    model.cuda() # move model to GPU, if available

model.train()

for e in range(epochs):
  hidden_state = model.init_hidden(batch_size)     # initialize hidden state

  for inputs, labels in train_loader:
      counter += 1
      if(device):
          inputs, labels = inputs.cuda(), labels.cuda()
      hidden_state = tuple([each.data for each in hidden_state])

      model.zero_grad()
      output, hidden_state = model(inputs, hidden_state)
      loss = model_loss(output.squeeze(), labels.float())
      loss.backward()
      nn.utils.clip_grad_norm_(model.parameters(), gradient_clip)
      optimizer.step()

      if counter % print_every == 0:
          
          val_hidden_state = model.init_hidden(batch_size) # Get validation loss
          val_losses = []
          model.eval()
          for inputs, labels in valid_loader:

              val_hidden_state = tuple([each.data for each in val_hidden_state])

              if(device):
                  inputs, labels = inputs.cuda(), labels.cuda()

              output, val_hidden_state = model(inputs, val_hidden_state)
              val_loss = model_loss(output.squeeze(), labels.float())

              val_losses.append(val_loss.item())
          
          print("Epoch: {}/{}:".format(e+1, epochs),
                "\t Loss: {:.5f}:".format(loss.item()))
# plt.plot(val_losses)
# plt.show() 

Epoch: 1/10: 	 Loss: 0.41127:
Epoch: 1/10: 	 Loss: 0.36212:
Epoch: 2/10: 	 Loss: 0.13671:
Epoch: 2/10: 	 Loss: 0.16968:
Epoch: 3/10: 	 Loss: 0.10537:
Epoch: 3/10: 	 Loss: 0.31508:
Epoch: 4/10: 	 Loss: 0.05865:
Epoch: 4/10: 	 Loss: 0.14510:
Epoch: 5/10: 	 Loss: 0.12971:
Epoch: 5/10: 	 Loss: 0.02323:
Epoch: 5/10: 	 Loss: 0.19623:
Epoch: 6/10: 	 Loss: 0.05141:
Epoch: 6/10: 	 Loss: 0.16443:
Epoch: 7/10: 	 Loss: 0.01325:
Epoch: 7/10: 	 Loss: 0.02685:
Epoch: 8/10: 	 Loss: 0.02043:
Epoch: 8/10: 	 Loss: 0.04405:
Epoch: 9/10: 	 Loss: 0.00413:
Epoch: 9/10: 	 Loss: 0.05652:
Epoch: 9/10: 	 Loss: 0.08037:
Epoch: 10/10: 	 Loss: 0.01336:
Epoch: 10/10: 	 Loss: 0.03484:


In [0]:
test_losses = [] # track loss
num_correct = 0

# init hidden state
hidden_state = model.init_hidden(batch_size)

model.eval()
# iterate over test data
for inputs, labels in test_loader:
  hidden_state = tuple([each.data for each in hidden_state])

  if(device):
      inputs, labels = inputs.cuda(), labels.cuda()
  
  output, hidden_state = model(inputs, hidden_state)
  
  test_loss = model_loss(output.squeeze(), labels.float())
  test_losses.append(test_loss.item())
  
  pred = torch.round(output.squeeze())  
  correct_tensor = pred.eq(labels.float().view_as(pred))
  correct = np.squeeze(correct_tensor.numpy()) if not device else np.squeeze(correct_tensor.cpu().numpy())
  num_correct += np.sum(correct)

print("Test loss: {:.3f}".format(np.mean(test_losses)))

test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))
# plt.plot(test_losses)
# plt.show()

Test loss: 0.790
Test accuracy: 0.820


In [0]:
def tokenize_review(test_review):
    test_review = test_review.lower() # lowercase
    # get rid of punctuation
    test_text = ''.join([c for c in test_review if c not in punctuation])

    # splitting by spaces
    test_words = test_text.split()
    
    # get rid of web address, twitter id, and digit
    new_text = []
    for word in test_words:
        if (word[0] != '@') & ('http' not in word) & (~word.isdigit()):
            new_text.append(word)

    # tokens
    test_ints = []
    test_ints.append([vocab_to_int[word] for word in new_text])

    return test_ints

In [0]:
def predict(model, test_review, sequence_length=30):
    
    model.eval()

    test_ints = tokenize_review(test_review)
    seq_length=sequence_length
    features = padding(test_ints, seq_length)

    feature_tensor = torch.from_numpy(features)
    
    batch_size = feature_tensor.size(0)

    hidden_state = model.init_hidden(batch_size)
    
    if(device):
        feature_tensor = feature_tensor.cuda()
    output, hidden_state = model(feature_tensor, hidden_state)

    pred = torch.round(output.squeeze()) 
    print('Prediction value: {:.12f}'.format(output.item()))
    
    # print custom response
    if(pred.item()==1):
        print("Positive review.")
    else:
        print("Negative review.")

In [0]:
seq_length = 30
test_review = "@AmericanAir you have my money, you change my flight, and don't answer your phones! Any other suggestions so I can make my commitment??"
predict(model, test_review, seq_length)

Prediction value: 0.045285705477
Negative review.


In [0]:
test_text = "@VirginAmerica I love the hipster innovation. You are a feel good brand."


In [0]:
predict(model, test_text, seq_length)

Prediction value: 0.999496698380
Positive review.


In [0]:
test_review2 = "@united  Delay DEN-CLE because they have to manually enter baggage tags? Really? Worst cust service day for this 1ker. #friendlyskies??"

In [0]:
predict(model, test_review2, seq_length)

Prediction value: 0.000030525101
Negative review.


In [0]:
test_review3 = "@United an idea: monitor mileage members travel patterns enough to know when they change jobs/lose status...and make the transition easier."

In [0]:
predict(model,test_review3,seq_length)

Prediction value: 0.999949097633
Positive review.


# **Working on my data**
My data consist of 100 positive and 100 negative teewts and it have an accuracy of 80%.

**Model Strength And Weakness**

Strength of my model is it have good accuracy and it is predicting the sentiments quite well.
Weakness that I found is that loss is not linear i.e. it is very fluctuating. Also the dataset I worked on was not quite big so if there is word out of the vocabulary then it is throwing an error. 


In [0]:
# import torch
# model = torch.load('/content/drive/My Drive/Colab Notebooks/model')
# model.eval()
"""I have preprocessed my own data during preprocessing the orignal data"""
valid_losses = [] # track loss
num_correct = 0

# init hidden state
hidden_state = model.init_hidden(batch_size)

model.eval()
# iterate over test data
for inputs, labels in valid_loader:
  hidden_state = tuple([each.data for each in hidden_state])

  if(device):
      inputs, labels = inputs.cuda(), labels.cuda()
  
  output, hidden_state = model(inputs, hidden_state)
  
  test_loss = model_loss(output.squeeze(), labels.float())
  test_losses.append(test_loss.item())
  
  pred = torch.round(output.squeeze())  
  correct_tensor = pred.eq(labels.float().view_as(pred))
  correct = np.squeeze(correct_tensor.numpy()) if not device else np.squeeze(correct_tensor.cpu().numpy())
  num_correct += np.sum(correct)

print("My data loss: {:.3f}".format(np.mean(test_losses)))

test_acc = num_correct/len(test_loader.dataset)
print("My data accuracy: {:.3f}".format(test_acc))

My data loss: 0.852
My data accuracy: 0.802


In [0]:
torch.save(model.state_dict(), '/content/drive/My Drive/Colab Notebooks/model')

Reference:

[1] https://towardsdatascience.com/sentiment-analysis-on-us-twitter-airline-dataset-1-of-2-2417f204b971

[2] https://colab.research.google.com/github/agungsantoso/deep-learning-v2-pytorch/blob/master/sentiment-rnn/Sentiment_RNN_Exercise.ipynb

[3] Dataset Reference: https://www.kaggle.com/crowdflower/twitter-airline-sentiment#Tweets.csv